In [ ]:
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from functools import reduce
import _pickle as pickle

In [ ]:
eng_rus = {'q':'й', 'w':'ц', 'e':'у', 'r':'к', 't':'е', 'y':'н'\
           , 'u':'г', 'i':'ш', 'o':'щ', 'p':'з','[':'х',']':'ъ','a':'ф',\
           's':'ы','d':'в','f':'а','g':'п','h':'р','j':'о','k':'л'\
           ,'l':'д',';':'ж','\'':'э','z':'я','x':'ч','c':'с','v':'м','b':'и','n':'т','m':'ь',',':'б','.':'ю'}
rus_eng = {rus:eng for eng, rus in eng_rus.items()}

In [ ]:
def flatten(x):
    result = []
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str) and not isinstance(elem, tuple) and len(elem)>1:
            result.extend(flatten(elem))
        else:
            result.append(elem)
    return result

In [ ]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [ ]:
def combinations(params):
    
    list_ = params
    tmp = reduce(lambda x, y: list_concat(x, y), list_)
    
    res = []
    for elem in tmp:
        param_list = flatten([elem])
        res.append(param_list)
        
    return res

In [ ]:
def corrected_requests():
    req_corr=[]
    req=[]
    corr=[]
    with open('queries_all.txt', 'r') as f:
        line = f.readline()
        while line:
            if line.find('\t') != -1:
                tmp = line.split('\t')
                s1 = tmp[0].split()
                s2 = tmp[1].split()
                if len(s1) == len(s2):
                    for i in range(len(s1)):
                        if s1[i] != s2[i] and len(s1[i]) < 14:
                            req += [s1[i].lower()]
                            corr += [s2[i].lower()]
            line = f.readline()
    req_corr = list(zip(req,corr))
    return req_corr
l_ = corrected_requests()

In [ ]:
def levenshtein(seq1, seq2):
   
    size_x = len(seq1)+1
    size_y = len(seq2)+1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix[x, y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1)
            else:
                matrix[x, y] = min(
                    matrix[x-1, y] + 1,     # удаление
                    matrix[x-1, y-1] + 1,   # замена
                    matrix[x, y-1] + 1)     # вставка

    return matrix#[size_x-1, size_y-1]

In [ ]:
def optimumWalk():
    
    delete_dict = {}
    replace_dict = {}
    insert_dict = {}
    
    def correctWalk(x=0, y=0, seq1='', seq2=''):
        
        if not seq1[:x-1] and not seq2[:y-1]:
            return 1

        res = 0
        
        
        if seq1[:x-1] and seq2[:y-1]:
            
            # Удаление оптимально 
            if matrix[x-1, y] <= matrix[x-1, y-1] and matrix[x-1, y] <= matrix[x, y-1]:
                
                if seq1[x:] and (seq2[y-1] != seq1[x-1] or seq1[x-1] == seq1[x-2]):
                    if delete_dict.get(seq1[x-1]):
                        if delete_dict[seq1[x-1]].get(seq1[x-2]):
                            delete_dict[seq1[x-1]][seq1[x-2]] += 1
                        else:
                            delete_dict[seq1[x-1]][seq1[x-2]] = 1
                    else:
                        delete_dict[seq1[x-1]] = {seq1[x-2] : 1}

                res += correctWalk(x-1, y, seq1, seq2)
                
            # Замена оптимальна
            if matrix[x-1, y-1] <= matrix[x-1, y] and matrix[x-1, y-1] <= matrix[x, y-1]:
                
                if seq1[x:] and (seq1[x-1] != seq2[y-1]):
                    if replace_dict.get(seq2[y-1]):
                        if replace_dict[seq2[y-1]].get(seq1[x-1]):
                            if replace_dict[seq2[y-1]][seq1[x-1]].get(seq1[x-2]):
                                replace_dict[seq2[y-1]][seq1[x-1]][seq1[x-2]] += 1
                            else:
                                replace_dict[seq2[y-1]][seq1[x-1]][seq1[x-2]] = 1
                        else:
                            replace_dict[seq2[y-1]][seq1[x-1]] = {seq1[x-2] : 1}
                    else:
                        replace_dict[seq2[y-1]] = {seq1[x-1] : {seq1[x-2] : 1}}
                        
                res += correctWalk(x-1, y-1, seq1, seq2)
                
            # Вставка оптимальна
            if matrix[x, y-1] <= matrix[x-1, y-1] and matrix [x, y-1] <= matrix[x-1, y]:
               
                if seq2[y:] and (seq1[x-1] != seq2[y-1] or seq2[y-1] == seq2[y-2]):
                    if insert_dict.get(seq2[y-1]):
                        if insert_dict[seq2[y-1]].get(seq1[x-1]):
                            insert_dict[seq2[y-1]][seq1[x-1]] += 1
                        else:
                            insert_dict[seq2[y-1]][seq1[x-1]] = 1
                    else:
                        insert_dict[seq2[y-1]] = {seq1[x-1] : 1}
                        
                res += correctWalk(x, y-1, seq1, seq2)

        # Безусловная вставка(дошли до края по x) 
        if not seq1[:x-1]:
            if seq2[y:] and (seq2[y-1] == seq2[y-2]):
                if insert_dict.get(seq2[y-1]):
                    if insert_dict[seq2[y-1]].get(seq1[x-1]):
                        insert_dict[seq2[y-1]][seq1[x-1]] += 1
                    else:
                        insert_dict[seq2[y-1]][seq1[x-1]] = 1
                else:
                    insert_dict[seq2[y-1]] = {seq1[x-1] : 1}
            
            correctWalk(x, y-1, seq1, seq2)

        # Безусловное удаление(дошли до края по y)
        if not seq2[:y-1]:
            if seq1[x:] and (seq2[y-1] != seq1[x-1] or seq1[x-1] == seq1[x-2]):
                if delete_dict.get(seq1[x-1]):
                    if delete_dict[seq1[x-1]].get(seq1[x-2]):
                        delete_dict[seq1[x-1]][seq1[x-2]] += 1
                    else:
                        delete_dict[seq1[x-1]][seq1[x-2]] = 1
                else:
                    delete_dict[seq1[x-1]] = {seq1[x-2] : 1}

            correctWalk(x-1, y, seq1, seq2)

        return res
    
    
    list_ = corrected_requests()
    
    for i in range(len(list_)):
        seq1 = '*' + list_[i][0] + '*'
        seq2 = '*' + list_[i][1] + '*'
        matrix = levenshtein(seq1, seq2)
        correctWalk(matrix.shape[0]-1, matrix.shape[1]-1, seq1, seq2)
    
    # Получаем частоты
    s=0
    for key1 in delete_dict:
        for key2 in delete_dict[key1]:
            s += delete_dict[key1][key2]

    for key1 in delete_dict:
        for key2 in delete_dict[key1]:
            delete_dict[key1][key2] /= s
            
    s=0
    for key1 in replace_dict:
        for key2 in replace_dict[key1]:
            for key3 in replace_dict[key1][key2]:
                s += replace_dict[key1][key2][key3]
    
    for key1 in replace_dict:
        for key2 in replace_dict[key1]:
            for key3 in replace_dict[key1][key2]:
                replace_dict[key1][key2][key3] /= s
    s=0
    for key1 in insert_dict:
        for key2 in insert_dict[key1]:
            s += insert_dict[key1][key2]
            
    for key1 in insert_dict:
        for key2 in insert_dict[key1]:
            insert_dict[key1][key2] /= s
        
    return delete_dict, replace_dict, insert_dict

In [ ]:
class SimpleStorage:
    def __init__(self):
        self.fileName = 'queries_all.txt'
        self.corpus = []
        self.freqWords = {}
        self.freqBigramsWords = {}
        
    def initFreqWords(self):
        s = 0
        
        with open(self.fileName, 'r') as f:
            for word in f.read().split():
                word = word[:20]
                word = word.lower()
                
                if word not in self.freqWords:
                    self.freqWords[word] = 1
                else:
                    self.freqWords[word] += 1
                    
        for key in list(self.freqWords):
            if self.freqWords[key] < 3:
                del self.freqWords[key]

        for key in self.freqWords:
            s += self.freqWords[key]
            
        for key in self.freqWords:
            self.freqWords[key] /= s
            
            
    def initCorpus(self):
        with open(self.fileName, 'r') as f:
            line = f.readline()
            while line:
                self.corpus += [line[:-1].lower()]
                line = f.readline()


    def initBigramsWords(self):
        s = 0
        vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))

        X = vectorizer.fit_transform(self.corpus)
        bigramms = vectorizer.get_feature_names()
        freqs = np.array(X.sum(axis=0))[0]

        for i in range(freqs.shape[0]):
            if freqs[i] > 1:
                self.freqBigramsWords[bigramms[i]] = freqs[i]
        
        for key in self.freqBigramsWords:
            s += self.freqBigramsWords[key]
            
        for key in self.freqBigramsWords:
            self.freqBigramsWords[key] /= s
            
    def getWords(self):
        return self.freqWords
    
    def getBigrams(self):
        return self.freqBigramsWords
    

In [ ]:
def searchRequest(request):
    
    if not request:
        return "Задан пустой поисковый запрос"
    request = request.lower()
    l = request.split(' ')
    l = [i.replace(' ', '') for i in l]
    l = [i for i in l if i]
    candidates = []
    prop = []
    propW = {}
    flag = False
    tmp1 = []
    max_ = -1
    
    #  Разделение   
    for string in l:
        if My_Tree.search_relevant(string) == []:
            for i in range(len(string)):
                if string[:i+1] in STORAGE.getWords() and string[i+1:] in STORAGE.getWords():
                    if STORAGE.getWords()[string[:i+1]] + STORAGE.getWords()[string[i+1:]] > max_:
                        tmp1 += [string[:i+1]]
                        tmp1 += [string[i+1:]]
                        max_ = STORAGE.getWords()[string[:i+1]] + STORAGE.getWords()[string[i+1:]]
            if max_ == -1:
                tmp1 += [string]
            else:
                max_ = -1
        else:
            tmp1 += [string]
                        
    l = tmp1
    # ____________________________________________________________________________________________________________

    # Соединение
    tmp2=[]
    flag = True
    
    for i in range(len(l)-1):
        
        if l[i]+l[i+1] in STORAGE.getWords():
            tmp2 = tmp2[:i] + [l[i]+l[i+1]]
            flag = False
        
        else:
            if flag:
                tmp2 += [l[i]]
            if i+2 == len(l):
                tmp2 += [l[-1]]
            flag = True
        
    # ____________________________________________________________________________________________________________

    if tmp2:
        l = tmp2
        
    for string in l:
        tmp = My_Tree.search_relevant(string)
        if tmp:
            tmp.sort(key = lambda x: (x[1]))
            tmp = tmp[-5:]
            candidates += [tmp]
        
    tmp = []
    for i in range(len(candidates)):
        tmp += [[j[0] for j in candidates[i]]]
        
        for j in candidates[i]:
            propW[j[0]] = j[1]

    if tmp:
        variables = combinations(tmp)
    else:
        return request
    tmp = 0
    for i in range(len(variables)):
        for j in range(len(variables[i])-1):
            if STORAGE.getBigrams().get(variables[i][j]+' '+variables[i][j+1]):
                tmp += np.log(STORAGE.getBigrams()[variables[i][j]+' '+variables[i][j+1]])
            else:
                tmp += 0
        prop += [tmp]
        tmp = 0
        
    tmp = 1
    for i in range(len(prop)):
        for j in range(len(variables[i])):
            tmp += propW[variables[i][j]]
        prop[i] += tmp
        tmp = 1
    
    prop.sort()
    ind = np.argsort(prop)    
    result = [variables[i] for i in ind[-1:]]
    result = result[0]
    out = ''
    for i in range(len(result)):
        out += result[i]+' '
        
    return out[:-1]

In [ ]:
class Trie:
    
    def __init__(self):
        self.children = {}
        self.word = ''
        self.isWord = False
                
    def insert(self,  word, level=0):
        """
        Recursion insert of new word in Trie
        """
        if not word:
            return

        if word[level] not in self.children:
            self.children[word[level]] = Trie()
        
        
        if word[level+1:]:
            self.children[word[level]].insert(word, level+1)
            
        else:
            self.children[word[level]].word = word
            self.children[word[level]].isWord = True
        
    def search_relevant(self, word, level=0, p=0.9999, translate = True, insert=False, delete=False):
        """
        Returns list of closest words in Trie
        1 - 0.45197
        2 - 0.20419
        3 - 0.09227
        4 - 0.04169
        5 - 0.01884
        *10**(-5)
        """        
        
        if p < 0.04*(10**-5):
            return []
        
        if not word[level:]:
            if self.isWord:
                return [(self.word, 1*np.log(STORAGE.getWords()[self.word])+np.log(p),np.log(p))]
            else:
                return []
        
        res = []
        
        # Раскладка
        if translate:
            f = 0
            new_word = ''
            for i in word:
                if not i in eng_rus:
                    f = 1
                else:
                    new_word += eng_rus[i]
                    
            if not f:
                res += self.search_relevant(new_word,0,0.9999,False)
            
            f = 0
            new_word = ''
            for i in word:
                if not i in rus_eng:
                    f = 1
                else:
                    new_word += rus_eng[i]
            if not f:
                res += self.search_relevant(new_word,0,0.9999,False)
        
        
        for letter in self.children.keys():

            # Вставка
            if letter in INS_D and not delete:
                if word[level-1] in INS_D[letter]:
                    res += self.children[letter].search_relevant(word, level, p*INS_D[letter][word[level-1]],\
                                                                 False, False, True)

            # Идём по слову
            if word[level] == letter:
                res += self.children[letter].search_relevant(word, level+1, p, False, False, False)
            
            # Замена
            else:
                if word[level] in REPL_D:
                    if letter in REPL_D[word[level]]:
                        if word[level-1] in REPL_D[word[level]][letter]:
                            res += self.children[letter].search_relevant(word, level+1,\
                                                                        p*REPL_D[word[level]][letter][word[level-1]],\
                                                                        False, False, False)
            # Удаление
            if word[level] in DEL_D and (not insert) and (not delete):
                if word[level-1] in DEL_D[word[level]]:
                    res += self.children[letter].search_relevant(word[:level]+word[level+1:], level+1,\
                                    p*DEL_D[word[level]][word[level-1]], False, True, False)
                    
            
        return res
    
    def isPrefix(self, prefix):
        """
        Returns True if the word  is prefix
        """
        
        if not prefix:
            return True
        
        if prefix[0] in self.children:
            return self.children[prefix[0]].isPrefix(prefix[1:])
        
        return False

In [ ]:
%%time
"""
Инициализация словаря слов, словаря биграмм слов.
"""

STORAGE = SimpleStorage()
STORAGE.initFreqWords()
STORAGE.initCorpus()
STORAGE.initBigramsWords()

CPU times: user 36.3 s, sys: 790 ms, total: 37.1 s
Wall time: 37.1 s


In [ ]:
%%time
"""
Инициализация словарей ошибок, таких как: удаление, замена, вставка
"""
DEL_D, REPL_D, INS_D = optimumWalk()

CPU times: user 35.7 s, sys: 371 ms, total: 36.1 s
Wall time: 34.8 s


In [ ]:
%%time
"""
Инициализация бора, где будут храниться все слова из словаря слов
"""
My_Tree = Trie()
for i in STORAGE.getWords():
    My_Tree.insert(i)

CPU times: user 4.76 s, sys: 9.18 ms, total: 4.77 s
Wall time: 4.76 s


In [ ]:
%%time
"""
Пример поиска кандидатов одного слова
"""
result = My_Tree.search_relevant('смоаfреть')
result.sort(key = lambda x: (x[1]))
result[::-1][:5]

CPU times: user 21.3 ms, sys: 23 µs, total: 21.3 ms
Wall time: 19.7 ms


[('смотреть', -13.474376718361565, -9.285332007004758),
 ('смоьреть', -23.311314840347492, -9.285332007004758),
 ('ссмотреть', -27.895240153682177, -13.686935763545488)]

In [ ]:
%%time
result = My_Tree.search_relevant('молоооко')
result.sort(key = lambda x: (x[1]))
result[::-1][:5]

CPU times: user 39.6 ms, sys: 0 ns, total: 39.6 ms
Wall time: 37.3 ms


[('молочко', -15.97251732515777, -3.373650847455183),
 ('молоко', -21.543202126299683, -11.409039591067573),
 ('молодого', -22.87115923354383, -10.92461794188093),
 ('молочка', -23.83572229432051, -9.404274352869612),
 ('молочко', -24.649759954942557, -12.050893477239969)]

In [ ]:
%%time
result = My_Tree.search_relevant('прзенация')
result.sort(key = lambda x: (x[1]))
result[::-1][:5]

CPU times: user 27.8 ms, sys: 207 µs, total: 28 ms
Wall time: 26.1 ms


[('презентация', -21.916710719143445, -13.14387353370203),
 ('призентация', -26.396336391335105, -13.574326362318306)]

In [ ]:
%%time
"""
Пример поиска кандидатов запроса
"""
searchRequest('смоаfреть ghptynfwb')

CPU times: user 81.1 ms, sys: 4 µs, total: 81.1 ms
Wall time: 78.5 ms


'смотреть презентации'

In [ ]:
%%time
searchRequest('скчать игаы нлайн')

CPU times: user 167 ms, sys: 0 ns, total: 167 ms
Wall time: 164 ms


'скачать игры онлайн'

In [ ]:
%%time
searchRequest('ку пить домашнийтелевизор')

CPU times: user 135 ms, sys: 76 µs, total: 135 ms
Wall time: 132 ms


'купить домашний телевизор'

In [ ]:
"""
Скрипты для сохранения моделей
"""
# STORAGE = SimpleStorage()
# STORAGE.initFreqWords()
# STORAGE.initCorpus()
# STORAGE.initBigramsWords()

# output = open("STORAGE.pkl", "wb")
# pickle.dump(STORAGE, output, 2)
# output.close()


# DEL_D, REPL_D, INS_D = optimumWalk()

# output = open("DEL_D.pkl", "wb")
# pickle.dump(DEL_D, output, 2)
# output.close()

# output = open("REPL_D.pkl", "wb")
# pickle.dump(REPL_D, output, 2)
# output.close()

# output = open("INS_D.pkl", "wb")
# pickle.dump(INS_D, output, 2)
# output.close()  

"""
Скрипты для загрузки моделей
"""
# My_Tree = Trie()
# for i in STORAGE.getWords():
#     My_Tree.insert(i)

# output = open("My_Tree.pkl", "wb")
# pickle.dump(My_Tree, output, 2)
# output.close()

# input = open('My_Tree.pkl','rb')
# My_Tree = pickle.load(input)
# input.close()

# #Глобальный словарь слов
# input = open('STORAGE.pkl','rb')
# STORAGE = pickle.load(input)
# input.close()

# input = open('DEL_D.pkl','rb')
# DEL_D = pickle.load(input)
# input.close()

# input = open('REPL_D.pkl','rb')
# REPL_D = pickle.load(input)
# input.close()

# input = open('INS_D.pkl','rb')
# INS_D = pickle.load(input)
# input.close()